# convect each results into excel

In [2]:
import pandas as pd

df = pd.read_csv('results/summarized_results.csv')

# Sort the dataframe

df_sorted = df.drop(columns=['nn_model', 'number_of_clients'])
# Sort the dataframe by skew, dataset, and heterogeneity_class

# Define the order and bold the 'cornflqs' exp_type
exp_type_order = ['fedavg', 'fedprox', 'cfl', 'hcfl', 'fedgroup', 'ifca', 'srfca', 'cornflqs', 'oracle-centralized']
df_sorted['exp_type'] = pd.Categorical(df_sorted['exp_type'], categories=exp_type_order, ordered=True)
# Add a newline before each occurrence of 'fedavg' in the 'exp_type' column
df_sorted['exp_type'] = df_sorted['exp_type'].apply(lambda x: '\n' + x if x == 'fedavg' else x)
# Sort the dataframe by the defined order
df_sorted = df_sorted.sort_values(['skew', 'dataset', 'heterogeneity_class','exp_type'])
df_sorted.to_excel('results.xlsx', index=False)

In [5]:
df=pd.read_csv('granular_results/noqs.csv')
# Define the order and bold the 'cornflqs' exp_type
df_sorted = df
exp_type_order = ['fedavg', 'fedprox', 'cfl', 'hcfl', 'fedgroup', 'ifca', 'srfca', 'cornflqs', 'oracle-centralized']
df_sorted['exp_type'] = pd.Categorical(df_sorted['exp_type'], categories=exp_type_order, ordered=True)
df_sorted['exp_type'] = df_sorted['exp_type'].apply(lambda x: '\n' + x if x == 'fedavg' else x)
df_sorted = df_sorted.sort_values(['skew', 'dataset', 'heterogeneity_class','exp_type'])
df_sorted.to_excel('noqs.xlsx', index=False)

In [7]:
df=pd.read_csv('/home/mbenali/Documents/Project/CFL-Evaluation-of-labels-and-quantity-skew/granular_results/qs1.csv')
df_sorted = df
exp_type_order = ['fedavg', 'fedprox', 'cfl', 'hcfl', 'fedgroup', 'ifca', 'srfca', 'cornflqs', 'oracle-centralized']
df_sorted['exp_type'] = pd.Categorical(df_sorted['exp_type'], categories=exp_type_order, ordered=True)
df_sorted['exp_type'] = df_sorted['exp_type'].apply(lambda x: '\n' + x if x == 'fedavg' else x)
df_sorted = df_sorted.sort_values(['skew', 'dataset', 'heterogeneity_class','exp_type'])
df_sorted.to_excel('qs1.xlsx', index=False)


In [6]:
df=pd.read_csv('granular_results/qs2.csv')
# Define the order and bold the 'cornflqs' exp_type
exp_type_order = ['fedavg', 'fedprox', 'cfl', 'hcfl', 'fedgroup', 'ifca', 'srfca', 'cornflqs', 'oracle-centralized']
df_sorted = df
df_sorted['exp_type'] = pd.Categorical(df_sorted['exp_type'], categories=exp_type_order, ordered=True)
df_sorted['exp_type'] = df_sorted['exp_type'].apply(lambda x: '\n' + x if x == 'fedavg' else x)
df_sorted = df_sorted.sort_values(['skew', 'dataset', 'heterogeneity_class','exp_type'])
df_sorted.to_excel('qs2.xlsx', index=False)

# calculate ranking 

In [7]:
def algo_ranking(dataset=None, skew=None, heterogeneity_class=None):
    import pandas as pd
    # Load the CSV data into a pandas DataFrame
    df = pd.read_csv('results/summarized_results.csv')
    if skew == 'quantity-skew-type-1' : 
        df = pd.read_csv('granular_results/qs1.csv')
        df.rename(columns={'global_accuracy': 'accuracy', 'homogeneity': 'hom', 'completeness': 'cmplt', 'v_measure': 'vm'}, inplace=True)
        granular_metrics = ['class_1_qt-skew_0.05', 'class_2_qt-skew_0.05', 'class_3_qt-skew_0.05', 'class_4_qt-skew_0.05',
                         'class_1_qt-skew_0.2', 'class_2_qt-skew_0.2', 'class_3_qt-skew_0.2', 'class_4_qt-skew_0.2',
                         'class_1_qt-skew_1', 'class_2_qt-skew_1', 'class_3_qt-skew_1', 'class_4_qt-skew_1',
                         'class_1_qt-skew_2', 'class_2_qt-skew_2', 'class_3_qt-skew_2', 'class_4_qt-skew_2']
        for metric in granular_metrics:
            df[metric] = df[metric].str.split(' ').str[0].astype(float)
        
    elif skew == 'quantity-skew-type-2' : 
        df = pd.read_csv('granular_results/qs2.csv')
        df.rename(columns={'global_accuracy': 'accuracy', 'homogeneity': 'hom', 'completeness': 'cmplt', 'v_measure': 'vm'}, inplace=True)
        granular_metrics= ['class_1_qt-skew_0.05', 'class_4_qt-skew_0.2', 'class_2_qt-skew_1', 'class_3_qt-skew_2']
        
        for metric in granular_metrics:
            df[metric] = df[metric].str.split(' ').str[0].astype(float)
     
    metrics = ['accuracy', 'ARI', 'AMI', 'hom', 'cmplt', 'vm']
    
    df.fillna('None', inplace=True)
    df = df[df['exp_type'] != 'oracle-centralized']
    df['exp_type'] = df['exp_type'] + '_' + df['params']
    
    # Filter the dataframe based on the input parameters
    if dataset:
        df = df[df['dataset'] == dataset]
    if skew:
        if skew == 'None':
            df = df[df['skew'] == skew]
        elif skew == 'quantity-skew':
            df = df[df['skew'] != 'None']
        else:
            df = df[df['skew'] == skew]
    if heterogeneity_class:
        df = df[df['heterogeneity_class'] == heterogeneity_class]

    # Step 2: Clean the data
    # Remove the '\pm' and uncertainty values from the 'accuracy' column
    df['accuracy'] = df['accuracy'].str.split(' ').str[0].astype(float)
    # Replace 'n/a' with NaN for missing values

    # Convert metric columns to numeric

    
    df[metrics] = df[metrics].apply(pd.to_numeric, errors='coerce')
    
    # Concatenate 'dataset', 'heterogeneity_class', and 'skew' into one column named 'group'
    df['group'] = df['dataset'] + '_' + df['heterogeneity_class'] + '_' + df['skew'] + '_' + df['seed'].astype(str)
    # Drop the original 'dataset', 'heterogeneity_class', and 'skew' columns
    df = df.drop(columns=['dataset', 'heterogeneity_class', 'skew', 'seed'])

    # Step 3: Group the data by the new 'group' column
    grouped = df.groupby('group')

    # Step 4: Rank algorithms within each group for each metric
    def rank_algorithms(group):
        for metric in metrics:
            group[f'{metric}_rank'] = group[metric].rank(ascending=False, method='min')
        return group

    ranked_groups = grouped.apply(rank_algorithms)
    best = ranked_groups.groupby('exp_type')[[f'{metric}_rank' for metric in metrics]].mean()
    best = best.sort_values(by='accuracy_rank')

    # Calculate the average accuracy for each exp_type
    avg_accuracy = ranked_groups.groupby('exp_type')['accuracy'].mean()
    best['avg_accuracy'] = avg_accuracy
    avg_ARI = ranked_groups.groupby('exp_type')['ARI'].mean()
    best['avg_ARI'] = avg_ARI
    # Calculate the average for each granular metric for each exp_type
    if skew in ['quantity-skew-type-1', 'quantity-skew-type-2']:
        for metric in granular_metrics:
            avg_metric = ranked_groups.groupby('exp_type')[metric].mean()
            best[f'{metric}'] = avg_metric
    return best


In [8]:
algo_ranking()

,accuracy_rank,ARI_rank,AMI_rank,hom_rank,cmplt_rank,vm_rank,avg_accuracy,avg_ARI
exp_type,,,,,,,,
cornflqs_1-1-20,3.102273,1.238636,1.215909,1.215909,1.022727,1.204545,80.015114,0.955795
cornflqs_1-10-10,3.155556,1.266667,1.233333,1.111111,1.222222,1.222222,79.558444,0.961333
ifca_best,3.177778,1.548148,1.555556,1.555556,1.296296,1.555556,78.614889,0.942074
hcfl_euclidean,3.555556,2.607407,2.600000,2.577778,2.659259,2.585185,76.697185,0.844370
fedgroup_edc,3.644444,2.259259,2.303704,2.266667,2.474074,2.311111,77.991556,0.874667
cornflqs_10-1-10,4.055556,1.933333,1.844444,1.622222,1.933333,1.888889,78.889556,0.892889
cfl_None,5.511111,4.881481,4.896296,4.822222,4.896296,4.896296,70.257111,0.560741
srfca_None,6.096296,2.081481,2.096296,2.133333,1.140741,2.140741,74.642963,0.831852
fedprox_0.1,7.822222,NaN,NaN,NaN,NaN,NaN,56.626889,NaN


In [9]:
algo_ranking(skew='quantity-skew-type-1')

,accuracy_rank,ARI_rank,AMI_rank,hom_rank,cmplt_rank,vm_rank,avg_accuracy,avg_ARI,class_1_qt-skew_0.05,class_2_qt-skew_0.05,...,class_3_qt-skew_0.2,class_4_qt-skew_0.2,class_1_qt-skew_1,class_2_qt-skew_1,class_3_qt-skew_1,class_4_qt-skew_1,class_1_qt-skew_2,class_2_qt-skew_2,class_3_qt-skew_2,class_4_qt-skew_2
exp_type,,,,,,,,,,,,,,,,,,,,,
cornflqs_1-1-20,2.372093,1.488372,1.441860,1.441860,1.046512,1.418605,80.524651,0.909535,80.037442,80.963256,...,79.547209,81.607674,80.037442,80.963256,79.393953,81.607674,80.037442,80.963256,79.470698,81.607674
cornflqs_1-10-10,2.911111,1.533333,1.466667,1.222222,1.444444,1.444444,79.473111,0.922667,79.032000,80.592000,...,77.180667,81.005778,79.032000,80.592000,77.235778,81.005778,79.032000,80.592000,77.466000,81.005778
cornflqs_10-1-10,3.133333,2.355556,2.200000,1.888889,2.244444,2.266667,78.492667,0.827778,75.964222,79.989333,...,77.362222,80.458000,75.964222,79.989333,77.616889,80.458000,75.964222,79.989333,77.918667,80.458000
ifca_best,3.177778,1.111111,1.088889,1.111111,1.133333,1.088889,79.542222,0.974667,79.519333,81.355111,...,78.416222,80.151556,77.365556,81.497111,76.420667,81.746889,78.624444,79.961556,77.850889,80.531778
fedgroup_edc,5.333333,3.933333,4.088889,4.022222,4.288889,4.111111,76.554667,0.732667,74.325333,74.772222,...,70.365556,77.286889,78.822222,79.713778,77.164222,80.716222,78.822222,79.554000,77.266222,80.469333
hcfl_euclidean,5.644444,5.422222,5.400000,5.377778,5.466667,5.355556,72.268000,0.573111,57.700889,64.310222,...,57.228667,80.780000,78.665556,80.235111,77.220000,80.691556,78.382667,80.163333,77.887778,80.826667
srfca_None,6.066667,1.377778,1.466667,1.422222,1.177778,1.466667,75.737111,0.919111,74.358667,76.931333,...,74.394889,77.419556,74.358667,76.956444,74.452444,77.419556,74.358667,76.956444,74.452444,77.324000
cfl_None,7.111111,6.600000,6.600000,6.600000,6.355556,6.600000,69.553556,0.428667,57.459111,68.353556,...,62.674000,75.319778,68.891556,77.087556,72.835111,75.696889,68.601333,77.121333,72.922889,75.798889
fedprox_0.1,8.800000,8.933333,8.933333,8.955556,8.955556,8.955556,58.138222,0.000000,58.138222,58.138222,...,58.138222,58.138222,58.138222,58.138222,58.138222,58.138222,58.138222,58.138222,58.138222,58.138222


In [10]:
algo_ranking(skew='quantity-skew-type-2')

,accuracy_rank,ARI_rank,AMI_rank,hom_rank,cmplt_rank,vm_rank,avg_accuracy,avg_ARI,class_1_qt-skew_0.05,class_4_qt-skew_0.2,class_2_qt-skew_1,class_3_qt-skew_2
exp_type,,,,,,,,,,,,
hcfl_euclidean,1.777778,1.000000,1.000000,1.000000,1.000000,1.000000,78.355333,1.000000,76.040444,80.697111,80.087333,76.593778
fedgroup_edc,1.911111,1.511111,1.488889,1.466667,1.711111,1.488889,77.876667,0.926889,75.206667,80.981778,79.797556,75.520222
ifca_best,2.577778,2.000000,2.044444,2.022222,1.755556,2.044444,76.897556,0.877333,74.242000,80.937556,79.816222,72.590889
srfca_None,4.044444,2.288889,2.244444,2.288889,1.111111,2.266667,73.802667,0.798222,68.111333,76.762222,75.620222,74.715778
cfl_None,4.866667,4.622222,4.666667,4.466667,4.688889,4.666667,64.653111,0.492222,44.012444,70.566222,73.154222,70.881111
fedprox_0.1,5.955556,5.888889,5.888889,6.000000,6.000000,6.000000,52.324222,0.000000,52.324222,52.324222,52.324222,52.324222
fedavg_None,6.844444,5.888889,5.888889,6.000000,6.000000,6.000000,50.598667,0.000000,50.598667,50.598667,50.598667,50.598667


In [11]:
algo_ranking(skew='quantity-skew')

,accuracy_rank,ARI_rank,AMI_rank,hom_rank,cmplt_rank,vm_rank,avg_accuracy,avg_ARI
exp_type,,,,,,,,
cornflqs_1-1-20,2.372093,1.488372,1.441860,1.441860,1.046512,1.418605,80.524651,0.909535
ifca_best,2.877778,1.555556,1.566667,1.566667,1.444444,1.566667,78.219889,0.926000
cornflqs_1-10-10,2.911111,1.533333,1.466667,1.222222,1.444444,1.444444,79.473111,0.922667
cornflqs_10-1-10,3.133333,2.355556,2.200000,1.888889,2.244444,2.266667,78.492667,0.827778
fedgroup_edc,3.622222,2.722222,2.788889,2.744444,3.000000,2.800000,77.215667,0.829778
hcfl_euclidean,3.711111,3.211111,3.200000,3.188889,3.233333,3.177778,75.311667,0.786556
srfca_None,5.055556,1.833333,1.855556,1.855556,1.144444,1.866667,74.769889,0.858667
cfl_None,5.988889,5.611111,5.633333,5.533333,5.522222,5.633333,67.103333,0.460444
fedprox_0.1,7.377778,NaN,NaN,NaN,NaN,NaN,55.231222,NaN


In [12]:
algo_ranking(skew='None')

,accuracy_rank,ARI_rank,AMI_rank,hom_rank,cmplt_rank,vm_rank,avg_accuracy,avg_ARI
exp_type,,,,,,,,
hcfl_euclidean,3.244444,1.400000,1.400000,1.355556,1.511111,1.400000,79.468222,0.960000
cornflqs_1-10-10,3.400000,1.000000,1.000000,1.000000,1.000000,1.000000,79.643778,1.000000
fedgroup_edc,3.688889,1.333333,1.333333,1.311111,1.422222,1.333333,79.543333,0.964444
ifca_best,3.777778,1.533333,1.533333,1.533333,1.000000,1.533333,79.404889,0.974222
cornflqs_1-1-20,3.800000,1.000000,1.000000,1.000000,1.000000,1.000000,79.528222,1.000000
cfl_None,4.555556,3.422222,3.422222,3.400000,3.644444,3.422222,76.564667,0.761333
cornflqs_10-1-10,4.977778,1.511111,1.488889,1.355556,1.622222,1.511111,79.286444,0.958000
srfca_None,8.177778,2.577778,2.577778,2.688889,1.133333,2.688889,74.389111,0.778222
fedprox_0.1,8.711111,NaN,NaN,NaN,NaN,NaN,59.418222,NaN


In [2]:
from utils_results import granular_results
granular_results("results/")

In [3]:
from utils_results import main_excel
main_excel()

Excel file created successfully!


# EXP CSV generation

In [ ]:
import pandas as pd
import itertools

# Define the list of tuples for exp_type and params
exp_params_list = [
    ('fedavg', 'None'),
    ('fedprox', 0.1),
    ('cfl', 'None'),
    ('hcfl', 'euclidean'),
    ('fedgroup', 'edc'),
    ('ifca', 'best'),
    ('srfca', 'None'),
    ('cornflqs', '1-10-10'),
    ('cornflqs', '10-1-10'),
    ('cornflqs', '1-1-20'),
    ('oracle-centralized', 'clustering')
]

# Define the base data
base_data = [
    ['mnist', 'concept-shift-on-features', 'quantity-skew-type-2'],
    ['mnist', 'concept-shift-on-labels', 'quantity-skew-type-2'],
    ['mnist', 'features-distribution-skew', 'quantity-skew-type-2'],
    ['fashion-mnist', 'concept-shift-on-features', 'quantity-skew-type-2'],
    ['fashion-mnist', 'concept-shift-on-labels', 'quantity-skew-type-2'],
    ['fashion-mnist', 'features-distribution-skew', 'quantity-skew-type-2'],
    ['kmnist', 'concept-shift-on-features', 'quantity-skew-type-2'],
    ['kmnist', 'concept-shift-on-labels', 'quantity-skew-type-2'],
    ['kmnist', 'features-distribution-skew', 'quantity-skew-type-2']
]

# Define fixed parameters
nn_model = 'linear'
number_of_clients = 100
num_samples_by_label = 50
num_clusters = 4
default_epochs = 10
rounds = 20
# Define the seed values
seed_values = [42, 43, 44, 45, 46]

# Create a list to store all the combinations
all_combinations = []

# Generate all combinations of base_data, exp_params_list, and seed_values
for base, (exp_type, params), seed in itertools.product(base_data, exp_params_list, seed_values):
    epochs = 100 if exp_type == 'oracle-centralized' else default_epochs
    combination = base + [exp_type, params, seed, nn_model, number_of_clients, num_samples_by_label, num_clusters, epochs, rounds]
    all_combinations.append(combination)

# Convert the list of combinations to a DataFrame
columns = ['dataset', 'heterogeneity_class', 'skew', 'exp_type', 'params', 'seed', 'nn_model', 'number_of_clients', 'num_samples_by_label', 'num_clusters', 'epochs', 'rounds']
df_combinations = pd.DataFrame(all_combinations, columns=columns)

# Reorder the columns as specified
df_combinations = df_combinations[['exp_type', 'params', 'dataset', 'nn_model', 'heterogeneity_class', 'skew', 'number_of_clients', 'num_samples_by_label', 'num_clusters', 'epochs', 'rounds', 'seed']]

# Save the DataFrame to a CSV file
df_combinations.to_csv('exp_configs.csv', index=False)